In [8]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout, interact, interactive, fixed, interact_manual
from IPython.display import clear_output

import matplotlib.pyplot as plt
from io import StringIO
plt.style.use('ggplot')
global my_file
global plt_title
global x_label
global y_label
global orientation
global fig_x
global fig_y
global fig_dpi
global line_width
global label_size
global bg_color
global title_color
global grid_color
global output

my_file = widgets.FileUpload(
    accept='*.txt',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description="Upload file:"
)

plt_title = widgets.Text(
    value='Allegro v1.2c (MultiPointPar) − Computation in sets of 100 markers, spacing 0.1000 cM',
    description="Plot Title:",
)
x_label = widgets.Text(
    value='Chromosomes',
    description="X axis label:",

)
y_label = widgets.Text(
    value='pLOD (MPT)',
    description="Y axis label:",

)

orientation = widgets.Dropdown(
    options=['landscape', 'portrait'],
    value='landscape',
    description='Orientation:',
)

fig_x = widgets.IntSlider(
    value=90,
    description="X axis width:",
)
fig_y = widgets.IntSlider(
    value=30,
    description="Y axis width:",
)
fig_dpi = widgets.IntSlider(
    value=60,
    description="Figue DPI:",

)
line_width = widgets.IntSlider(
    value=6,
    description="Line width:",

)
label_size = widgets.IntSlider(
    value=72,
    description="Label size:",
)

colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",
          "#8c564b", "#bcbd22", "#17becf", "#e377c2", "#7f7f7f"]
bg_color = widgets.ColorPicker(
    value='white',
    description='Background color:',
)
title_color = widgets.ColorPicker(
    value='black',
    description='Title color:',
)

grid_color = widgets.ColorPicker(
    value='black',
    description='Grid color:',
)
output = widgets.Output()
vbox = widgets.VBox([
    widgets.VBox([
        my_file,
        plt_title,
        x_label,
        y_label,
        orientation,
        fig_x,
        fig_y,
        fig_dpi,
        line_width,
        label_size,
        bg_color,
        title_color,
        grid_color,
    ], layout=Layout(width='100%')),
    widgets.HBox([
        output
    ], layout=Layout(width='100%'))
])
display(vbox)

def show_fig():
    content = list(my_file.value.values())[0]['content']
    io = StringIO(content.decode('utf-8'))

    df = pd.read_csv(io, delim_whitespace=True)
    df = df[['CHR', 'Position_cM', 'LOD']]
    df[df < .01] = .0001
    chromosomes = df[['CHR']].drop_duplicates(keep='first')['CHR'].tolist()

    df['ind'] = range(len(df))
    df_grouped = df.groupby(('CHR'))
    fig = plt.figure(figsize=(fig_x.value, fig_y.value), dpi=fig_dpi.value, constrained_layout=True)
    ax = fig.add_subplot(111)
    ax.set_facecolor(bg_color.value)
    
    x_labels = []
    x_labels_pos = []
    for num, (name, group) in enumerate(df_grouped):
        #     group.plot(kind='line', x='ind', y='LOD',color=colors[num % len(colors)], ax=ax, lw=1)
        line = group.plot(kind='line', y='LOD',
                   color=colors[num % len(colors)], ax=ax, lw=line_width.value)
        x_labels.append(int(name))
        x_labels_pos.append(
            (group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0]) / 2))
        ax.axvline(group['ind'].iloc[-1], linestyle='-', color='k', lw=4)
    ax.set_xticks(x_labels_pos)
    ax.set_xticklabels(x_labels)
    
    ax.set_xlim([0, len(df)])
    ax.set_ylim([0, 3])
    
    plt.title(plt_title.value, fontsize=label_size.value, color=title_color.value)
    ax.set_xlabel(x_label.value, fontsize=label_size.value)
    ax.set_ylabel(y_label.value, fontsize=label_size.value)
    ax.legend_.remove()
    ax.grid(False)
    
    ax.yaxis.grid(color=grid_color.value, linestyle='dotted', linewidth=1.5)
    
    plt.yticks(fontsize=label_size.value)
    plt.xticks(fontsize=label_size.value)
    widgets.interaction.show_inline_matplotlib_plots()


def update(change):
    with output:
        clear_output()
        show_fig()
    
my_file.observe(
    update,
    "value",
)

plt_title.observe(
    update,
    "value",
)
x_label.observe(
    update,
    "value",
)
y_label.observe(
    update,
    "value",
)
orientation.observe(
    update,
    "value",
)
fig_x.observe(
    update,
    "value",
)
fig_y.observe(
    update,
    "value",
)
fig_dpi.observe(
    update,
    "value",
)
line_width.observe(
    update,
    "value",
)
label_size.observe(
    update,
    "value",
)
bg_color.observe(
    update,
    "value",
)
title_color.observe(
    update,
    "value",
)
grid_color.observe(
    update,
    "value",
)